# Assessing the performance of the model

## Importing

In [21]:
import xarray as xr
import numpy as np
import pandas as pd

from sklearn.pipeline import make_pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate

from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor

from sklearn.metrics import root_mean_squared_error as rmse


## Datasets Preparation

In [22]:
def datasets_preparation(dataset, dataset2, name):
    
    x = np.tile(dataset.x, len(dataset.time_counter)*len(dataset.y))
    y = np.tile(np.repeat(dataset.y, len(dataset.x)), len(dataset.time_counter))
   
    inputs = np.stack([
        np.ravel(dataset2['Summation_of_solar_radiation']),
        np.ravel(dataset2['Mean_wind_speed']),
        np.ravel(dataset2['Mean_air_temperature']),
        y,
        x,
        np.repeat(dataset.time_counter.dt.dayofyear, len(dataset.x)*len(dataset.y)),
        ])

    targets = np.ravel(dataset[name])
    
    indx = np.where(np.isfinite(targets) & (x>10) & ((x>100) | (y<880)))
    inputs = inputs[:,indx[0]]
    targets = targets[indx[0]]

    inputs = inputs.transpose()

    return(inputs, targets, indx)


## Regressor (Training with all)

In [23]:
def regressor (inputs, targets, table):

    model = TransformedTargetRegressor(regressor=make_pipeline(ColumnTransformer(
        transformers=[('drivers', StandardScaler(), [0,1,2]), ],remainder='passthrough'),
        HistGradientBoostingRegressor(categorical_features=[3,4,5],learning_rate=0.5)),
        transformer=StandardScaler())
    regr = BaggingRegressor(model, n_estimators=12, n_jobs=4).fit(inputs,targets)
    
    predictions = regr.predict(inputs)
    
    table[0,0] = np.round(np.corrcoef(predictions,targets)[0][1],3)
    table[1,0] = rmse(predictions,targets)
    m,_ = np.polyfit(targets, predictions, deg=1)
    table[2,0] = np.round(m,3)

    return(regr)


## Regressor (Training with 75%, testing with 25%)

In [24]:
def regressor2 (inputs, targets, table):

    model = TransformedTargetRegressor(regressor=make_pipeline(ColumnTransformer(
        transformers=[('drivers', StandardScaler(), [0,1,2]), ],remainder='passthrough'),
        HistGradientBoostingRegressor(categorical_features=[3,4,5],learning_rate=0.5)),
        transformer=StandardScaler())
    
    X_train, X_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.25)
    regr = BaggingRegressor(model, n_estimators=12, n_jobs=4).fit(X_train,y_train)

    predictions = regr.predict(X_train)

    table[0,2] = np.round(np.corrcoef(y_train,predictions)[0][1],3)
    table[1,2] = rmse(y_train,predictions)
    m,_ = np.polyfit(y_train,predictions, deg=1)
    table[2,2] = np.round(m,3)
    
    predictions = regr.predict(X_test)

    table[0,3] = np.round(np.corrcoef(y_test,predictions)[0][1],3)
    table[1,3] = rmse(y_test,predictions)
    m,_ = np.polyfit(y_test,predictions, deg=1)
    table[2,3] = np.round(m,3)

    return(regr)


## Cross Validation (4 folds)

In [25]:
def regressor3 (inputs, targets, table):

    model = TransformedTargetRegressor(regressor=make_pipeline(ColumnTransformer(
        transformers=[('drivers', StandardScaler(), [0,1,2]), ],remainder='passthrough'),
        HistGradientBoostingRegressor(categorical_features=[3,4,5],learning_rate=0.5)),
        transformer=StandardScaler())
    
    regr = BaggingRegressor(model, n_estimators=12, n_jobs=4)

    predictions = cross_val_predict(regr,inputs,targets,cv=4)
    scores = cross_validate(regr, inputs, targets, cv=4, scoring=('r2', 'neg_root_mean_squared_error'), return_train_score=True)

    table[0,5:9] =  np.round(np.sqrt(np.abs(scores['train_r2'])),3)
    table[1,5:9] =  np.abs(scores['train_neg_root_mean_squared_error'])

    table[0,9:13] =  np.round(np.sqrt(np.abs(scores['test_r2'])),3)
    table[1,9:13] =  np.abs(scores['test_neg_root_mean_squared_error'])

    table[0,13] = np.round(np.corrcoef(predictions,targets)[0][1],3)
    table[1,13] = rmse(predictions,targets)
    m,_ = np.polyfit(targets, predictions, deg=1)
    table[2,13] = np.round(m,3)


## Evaluation (2021-2024)

In [26]:
def evaluation (ds,ds2,regr,name, table, i):

    dataset = ds.sel(time_counter = slice('2021', '2024'))
    dataset2 = ds2.sel(time_counter = slice('2021', '2024'))

    inputs, targets, _ = datasets_preparation(dataset, dataset2,name)

    predictions = regr.predict(inputs)

    table[0,i] = np.round(np.corrcoef(predictions,targets)[0][1],3)
    table[1,i] = rmse(predictions,targets)
    m,_ = np.polyfit(targets, predictions, deg=1)
    table[2,i] = np.round(m,3)


## Printing

In [27]:
def printing(table,criteria,categories,metric):

    temp = pd.DataFrame(table.transpose(),columns=criteria,index=categories)
    print(metric)
    display(temp)
    print ('\n')
    

## Training

In [28]:
def training(name,table):

    ds = xr.open_dataset('/data/ibougoudis/MOAD/files/integrated_original.nc')
    ds2 = xr.open_dataset('/data/ibougoudis/MOAD/files/external_inputs.nc')

    ds = ds.isel(time_counter = (np.arange(0, len(ds.time_counter),2)), 
        y=(np.arange(ds.y[0], ds.y[-1], 5)), 
        x=(np.arange(ds.x[0], ds.x[-1], 5)))

    ds2 = ds2.isel(time_counter = (np.arange(0, len(ds2.time_counter),2)), 
        y=(np.arange(ds2.y[0], ds2.y[-1], 5)), 
        x=(np.arange(ds2.x[0], ds2.x[-1], 5)))

    dataset = ds.sel(time_counter = slice('2007', '2020'))
    dataset2 = ds2.sel(time_counter = slice('2007', '2020'))

    inputs, targets, _ = datasets_preparation(dataset, dataset2, name)

    regr = regressor(inputs, targets, table)
    evaluation(ds,ds2,regr,name,table,1)

    regr2 = regressor2(inputs, targets, table)
    evaluation(ds,ds2,regr2,name,table,4)

    regressor3(inputs, targets, table)


## Main Body

In [29]:
criteria = ['r','rms','slope']
categories = ['training with 100%', 'testing', 'training with 75%', 'testing with 25%', 'testing', '1st fold train','2nd fold train', '3rd fold train','4th fold train',
    '1st fold test', '2nd fold test', '3rd fold test', '4th fold test', 'overall cross-val']

diat = np.zeros((len(criteria),len(categories)))
flag = np.zeros((len(criteria),len(categories)))
diat_pr = np.zeros((len(criteria),len(categories)))
flag_pr = np.zeros((len(criteria),len(categories)))

training('Diatom',diat)
training('Flagellate',flag)
training('Diatom_Production_Rate',diat_pr)
training('Flagellate_Production_Rate',flag_pr)


## Printing (Results)

In [30]:
printing(diat,criteria, categories,'Diatom')
printing(flag,criteria, categories,'Flagellate')
printing(diat_pr,criteria, categories,'Diatom production rate')
printing(flag_pr,criteria, categories, 'Flagellate production rate')


Diatom


,r,rms,slope
training with 100%,0.880,0.078988,0.709
testing,0.605,0.133332,0.460
training with 75%,0.878,0.079529,0.705
testing with 25%,0.860,0.084068,0.686
testing,0.604,0.133397,0.458
1st fold train,0.893,0.074225,0.000
2nd fold train,0.888,0.074964,0.000
3rd fold train,0.893,0.075695,0.000
4th fold train,0.906,0.067287,0.000
1st fold test,0.497,0.140308,0.000




Flagellate


,r,rms,slope
training with 100%,0.941,0.009378,0.857
testing,0.843,0.015476,0.721
training with 75%,0.941,0.009410,0.855
testing with 25%,0.934,0.009959,0.846
testing,0.844,0.015448,0.717
1st fold train,0.948,0.008635,0.000
2nd fold train,0.951,0.008738,0.000
3rd fold train,0.949,0.008800,0.000
4th fold train,0.951,0.008480,0.000
1st fold test,0.800,0.017591,0.000




Diatom production rate


,r,rms,slope
training with 100%,0.942,5.085883e-07,0.866
testing,0.871,7.471835e-07,0.807
training with 75%,0.942,5.107014e-07,0.865
testing with 25%,0.934,5.408370e-07,0.856
testing,0.872,7.450301e-07,0.806
1st fold train,0.946,4.935373e-07,0.000
2nd fold train,0.944,4.893619e-07,0.000
3rd fold train,0.948,4.761007e-07,0.000
4th fold train,0.953,4.674636e-07,0.000
1st fold test,0.875,7.199262e-07,0.000




Flagellate production rate


,r,rms,slope
training with 100%,0.950,1.259646e-07,0.883
testing,0.866,2.019906e-07,0.817
training with 75%,0.950,1.261386e-07,0.882
testing with 25%,0.943,1.334356e-07,0.873
testing,0.865,2.021165e-07,0.818
1st fold train,0.957,1.181360e-07,0.000
2nd fold train,0.955,1.206615e-07,0.000
3rd fold train,0.955,1.166412e-07,0.000
4th fold train,0.959,1.128696e-07,0.000
1st fold test,0.837,2.072029e-07,0.000
